**Preprocessing**

In [1]:
!pip uninstall -y numpy gensim
!pip install --no-cache-dir numpy==1.23.5 gensim

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 154.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 135.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 168.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 112.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.23.5 which is incompatible.
bigframes 2.3.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.6 requires numpy>=1.24.4, but you have numpy 1.2

In [1]:
import re
import numpy as np
import gensim.downloader as api
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

word2vec_model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [6]:
import pandas as pd
df = pd.read_csv('Combined Data.csv')
def preprocess_and_tokenize(text):
    if not isinstance(text, str):
        return []

    text = re.sub(r"[^A-Za-z0-9\s]", "", text.lower())
    tokens = simple_preprocess(text, deacc=True)
    tokens = [word for word in tokens if word not in ENGLISH_STOP_WORDS]
    return tokens

df['tokens'] = df['statement'].apply(preprocess_and_tokenize)
def get_average_word2vec(tokens, model, vector_size=300):
    vectors = [model[word] for word in tokens if word in model]
    return np.mean(vectors, axis=0) if vectors else np.zeros(vector_size)

df['word2vec_vector'] = df['tokens'].apply(lambda x: get_average_word2vec(x, word2vec_model))
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['status'] = le.fit_transform(df['status'])

label_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
print("Label Mapping:", label_mapping)

print(df['status'].value_counts())


Label Mapping: {'Anxiety': 0, 'Bipolar': 1, 'Depression': 2, 'Normal': 3, 'Personality disorder': 4, 'Stress': 5, 'Suicidal': 6}
status
3    16351
2    15404
6    10653
0     3888
1     2877
5     2669
4     1201
Name: count, dtype: int64


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import numpy as np

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Prepare data
X = np.stack(df['word2vec_vector'].values)
y = df['status'].values

X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.long).to(device)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [8]:
device

device(type='cuda')

TRANSFORMER + DT

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class TransformerFeatureExtractor(nn.Module):
    def __init__(self, input_size, d_model=128, num_heads=4, num_layers=2):
        super(TransformerFeatureExtractor, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=256, dropout=0.1, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        return x.mean(dim=1)

class SoftDecisionTree(nn.Module):
    def __init__(self, input_dim, num_classes, num_nodes=10):
        super(SoftDecisionTree, self).__init__()
        self.inner_nodes = nn.Linear(input_dim, num_nodes)
        self.leaf_nodes = nn.Linear(num_nodes, num_classes)

    def forward(self, x):
        decision_weights = torch.sigmoid(self.inner_nodes(x))
        leaf_outputs = self.leaf_nodes(decision_weights)
        return leaf_outputs

class TransformerDecisionTree(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_nodes=10):
        super(TransformerDecisionTree, self).__init__()
        self.feature_extractor = TransformerFeatureExtractor(input_dim, hidden_dim)
        self.decision_tree = SoftDecisionTree(hidden_dim, num_classes, num_nodes)

    def forward(self, x):
        features = self.feature_extractor(x)
        output = self.decision_tree(features)
        return output

class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).clone().detach().float()
        self.y = torch.tensor(y).clone().detach().long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

batch_size = 64
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train.shape[1]
num_classes = len(torch.unique(y_train))
model = TransformerDecisionTree(input_dim=input_dim, hidden_dim=128, num_classes=num_classes, num_nodes=10)

train_model(model, train_loader, test_loader, epochs=10, lr=0.001)


<ipython-input-12-f4c337697e3c>:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X).clone().detach().float()
<ipython-input-12-f4c337697e3c>:62: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).clone().detach().long()


Epoch 1, Loss: 1.4302, Train Accuracy: 47.65%
Epoch 2, Loss: 1.1343, Train Accuracy: 57.53%
Epoch 3, Loss: 1.0439, Train Accuracy: 60.91%
Epoch 4, Loss: 0.9940, Train Accuracy: 63.23%
Epoch 5, Loss: 0.9640, Train Accuracy: 64.37%
Epoch 6, Loss: 0.9422, Train Accuracy: 65.40%
Epoch 7, Loss: 0.9177, Train Accuracy: 66.16%
Epoch 8, Loss: 0.8973, Train Accuracy: 66.76%
Epoch 9, Loss: 0.8719, Train Accuracy: 67.93%
Epoch 10, Loss: 0.8630, Train Accuracy: 68.19%
Test Accuracy: 66.17%


TRANSFORMER + SVM

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class TransformerFeatureExtractor(nn.Module):
    def __init__(self, input_size, d_model=128, num_heads=4, num_layers=2):
        super(TransformerFeatureExtractor, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=256, dropout=0.1, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        return x.mean(dim=1)

class SoftSVM(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SoftSVM, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        return self.fc(x)

class TransformerSVM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(TransformerSVM, self).__init__()
        self.feature_extractor = TransformerFeatureExtractor(input_dim, hidden_dim)
        self.svm = SoftSVM(hidden_dim, num_classes)

    def forward(self, x):
        features = self.feature_extractor(x)
        output = self.svm(features)
        return output

class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).clone().detach().float()
        self.y = torch.tensor(y).clone().detach().long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

batch_size = 64
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train.shape[1]
num_classes = len(torch.unique(y_train))
model = TransformerSVM(input_dim=input_dim, hidden_dim=128, num_classes=num_classes)

train_model(model, train_loader, test_loader, epochs=10, lr=0.001)


<ipython-input-13-2e0227d7cd37>:58: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X).clone().detach().float()
<ipython-input-13-2e0227d7cd37>:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).clone().detach().long()


Epoch 1, Loss: 1.0705, Train Accuracy: 58.98%
Epoch 2, Loss: 0.8568, Train Accuracy: 67.56%
Epoch 3, Loss: 0.8068, Train Accuracy: 69.45%
Epoch 4, Loss: 0.7779, Train Accuracy: 70.33%
Epoch 5, Loss: 0.7586, Train Accuracy: 71.04%
Epoch 6, Loss: 0.7437, Train Accuracy: 71.63%
Epoch 7, Loss: 0.7319, Train Accuracy: 72.05%
Epoch 8, Loss: 0.7240, Train Accuracy: 72.41%
Epoch 9, Loss: 0.7145, Train Accuracy: 72.85%
Epoch 10, Loss: 0.7059, Train Accuracy: 73.01%
Test Accuracy: 71.80%


TRANSFORMER+RF

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1), :].to(x.device)

class TransformerFeatureExtractor(nn.Module):
    def __init__(self, input_size, d_model=128, num_heads=4, num_layers=2):
        super(TransformerFeatureExtractor, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads, dim_feedforward=256, dropout=0.1, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)

    def forward(self, x):
        x = self.embedding(x).unsqueeze(1)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        return x.mean(dim=1)

class SoftDecisionTree(nn.Module):
    def __init__(self, input_dim, num_classes, num_nodes=10):
        super(SoftDecisionTree, self).__init__()
        self.inner_nodes = nn.Linear(input_dim, num_nodes)
        self.leaf_nodes = nn.Linear(num_nodes, num_classes)

    def forward(self, x):
        decision_weights = torch.sigmoid(self.inner_nodes(x))
        leaf_outputs = self.leaf_nodes(decision_weights)
        return leaf_outputs

class SoftRandomForest(nn.Module):
    def __init__(self, input_dim, num_classes, num_trees=5, num_nodes=10):
        super(SoftRandomForest, self).__init__()
        self.trees = nn.ModuleList([SoftDecisionTree(input_dim, num_classes, num_nodes) for _ in range(num_trees)])

    def forward(self, x):
        tree_outputs = torch.stack([tree(x) for tree in self.trees], dim=0)
        return tree_outputs.mean(dim=0)

class TransformerRandomForest(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_trees=5, num_nodes=10):
        super(TransformerRandomForest, self).__init__()
        self.feature_extractor = TransformerFeatureExtractor(input_dim, hidden_dim)
        self.random_forest = SoftRandomForest(hidden_dim, num_classes, num_trees, num_nodes)

    def forward(self, x):
        features = self.feature_extractor(x)
        output = self.random_forest(features)
        return output

class CustomDataset(data.Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X).clone().detach().float()
        self.y = torch.tensor(y).clone().detach().long()

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def train_model(model, train_loader, test_loader, epochs=10, lr=0.001):
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

        train_acc = 100 * correct / total
        print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}, Train Accuracy: {train_acc:.2f}%")

    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

batch_size = 64
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

input_dim = X_train.shape[1]
num_classes = len(torch.unique(y_train))
model = TransformerRandomForest(input_dim=input_dim, hidden_dim=128, num_classes=num_classes, num_trees=5, num_nodes=10)

train_model(model, train_loader, test_loader, epochs=10, lr=0.001)


<ipython-input-14-4c201d129b25>:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X).clone().detach().float()
<ipython-input-14-4c201d129b25>:71: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).clone().detach().long()


Epoch 1, Loss: 1.3643, Train Accuracy: 49.65%
Epoch 2, Loss: 1.1066, Train Accuracy: 57.24%
Epoch 3, Loss: 1.0357, Train Accuracy: 60.78%
Epoch 4, Loss: 0.9737, Train Accuracy: 63.68%
Epoch 5, Loss: 0.9280, Train Accuracy: 65.21%
Epoch 6, Loss: 0.8947, Train Accuracy: 66.56%
Epoch 7, Loss: 0.8707, Train Accuracy: 67.55%
Epoch 8, Loss: 0.8509, Train Accuracy: 68.72%
Epoch 9, Loss: 0.8323, Train Accuracy: 69.04%
Epoch 10, Loss: 0.8126, Train Accuracy: 69.70%
Test Accuracy: 69.51%
